Imports.

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
from scipy.constants import physical_constants
import matplotlib.pyplot as plt
from diffpy.utils.parsers.loaddata import loadData
from bg_mpl_stylesheet.bg_mpl_stylesheet import bg_mpl_style

Dictionary containing sample and experimental information.

In [ ]:
D_SAMPLE = dict(mass_mg=6.755,
                molar_mass=79.866,
                x_start=0,
                voltage_min=1,
                voltage_max=3,
                )

Dictionary containing labels used in the header of the echem data file.

In [ ]:
D_ECHEM_LABELS = dict(time="t [h]", 
                      voltage="V [V]", 
                      current="i [A]",
                      )

Function to check whether code is run as a Ipython notebook or from an Ipython
terminal to be able to exit properly, if neccessary.

In [ ]:
def is_nb():
    shell = get_ipython().__class__.__name__
    if shell == "ZMQInteractiveShell":
        nb_bool = True
    else:
        nb_bool = False

    return nb_bool

Function to calculate the state of charge, $x$.

In [ ]:
def x_calculate(df, d):
    time, current = df["time"].to_numpy(), df["current"].to_numpy()
    molar_mass, mass = d["molar_mass"], d["mass_mg"] * 10**-3
    x = np.array([d["x_start"]])
    n = mass / molar_mass
    f = physical_constants["Faraday constant"][0]
    for i in range(1, len(time)):
        delta_q =  - current[i] * (time[i] - time[i-1]) * 60**2
        delta_x = delta_q / (n * f)
        x = np.append(x, x[-1] + delta_x)

    return x

Function to update keys in dataframe.

In [ ]:
def df_keys_update(df, d):
    for k1 in list(df.keys()):
        for k2 in D_ECHEM_LABELS:
            if D_ECHEM_LABELS[k2] in k1:
                df.rename(columns={k1: k2}, inplace=True)
    
    return df

Function to plot the voltage as a function of state of charge.

In [ ]:
def plot(df, d, filename, output_paths):
    figsize = (12, 4)
    dpi = 600
    fs_labels = 20
    fs_ticks = 14
    xlabel = "$x$"
    ylabel = "$V\;[\mathrm{h}]$"
    plt.style.use(bg_mpl_style)
    fig, ax = plt.subplots(figsize=figsize)
    ax.plot(df["x"], df["voltage"])
    ax.set_xlim(np.amin(df["x"]), np.amax(df["x"]))
    ax.set_ylim(d["voltage_min"], d["voltage_max"])
    ax.set_xlabel(xlabel, fontsize=fs_labels)
    ax.set_ylabel(ylabel, fontsize=fs_labels)
    ax.tick_params(axis="both", labelsize=fs_ticks)
    ax.minorticks_on()
    for p in output_paths:
        print(f"\t{p.name}")
        plt.savefig(p / f"{filename}.{p.name}", dpi=dpi, bbox_inches="tight")
    plt.close()

    return None

Function to write dataframe to `.txt` file in a format similar to the input.

In [ ]:
def write(df, d, filename, output_path):
    df_keys = df.keys()
    for k in df_keys:
        if k not in d.keys():
            d[k] = k
    delimiter="\t"
    header = d[df_keys[0]]
    for i in range(1, len(df_keys)):
        header += f"{delimiter}{d[df_keys[i]]}"
    np.savetxt(output_path / filename,
               df.to_numpy(),
               delimiter=delimiter,
               header=header,
               fmt="%.4e",
               encoding="utf8",
               )

    return None

Checking whether a folder called `data` exists.

In [ ]:
data_path = Path.cwd() / "data_echem"
if not data_path.exists():
    data_path.mkdir()
    s = f"{80*'-'}\nA folder called '{data_path.name}' has been created."
    s += f"\nPlease put your data files there and rerun the code.\n{80*'-'}"
    print(s)
    if is_nb():
        exit(keep_kernel=True)
    else:
        exit()

Checking that only one file i present in the `data` folder.

In [ ]:
data_files = list(data_path.glob("*.*"))
if len(data_files) == 0:
    s = f"{80*'-'}\nNo files were found in the '{data_path.name}' folder.\n"
    s += f"Please put your data files there and rerun the code.\n{80*'-'}"
    print(s)
    if is_nb():
        exit(keep_kernel=True)
    else:
        exit()
elif len(data_files) > 1:
    s = f"{80*'-'}\nMore than one file was found in the "
    s += f"'{data_path.name}' folder.\nPlease put your single data file there "
    s += f"and rerun the code.\n{80*'-'}"
    print(s)
    if is_nb():
        exit(keep_kernel=True)
    else:
        exit()
else:
    data_file = data_files[0]

Creating output paths if not already existing.

In [ ]:
output_folders = ["png", "pdf", "svg", "txt"]
output_paths = [Path.cwd() / folder for folder in output_folders]
plot_folders = [e for e in output_folders if not e == "txt"]
plot_paths = [p for p in output_paths if not p.name == "txt"]
for p in output_paths:
    if not p.exists():
        p.mkdir()
txt_path = Path.cwd() / "txt"

Reading data into Pandas dataframe (df), renaming dataframe keys, calculating 
state of charge ($x$) and adding it to df before plotting and writing to `.txt`
file.

In [ ]:
print(f"{80*'-'}\nSample and experimental information used...")
for k, v in D_SAMPLE.items():
      if len(k) < 8:
            print(f"\t{k}\t\t{v}")
      else:
            print(f"\t{k}\t{v}")
print(f"{80*'-'}\nWorking on: {data_file.name}\n{80*'-'}\nLoading data into "
      f"dataframe...")
df = pd.read_csv(data_file, delimiter="\t", header=0)
print(f"Done loading data into dataframe.\n{80*'-'}\nUpdating dataframe "
      f"keys...")
df = df_keys_update(df, D_ECHEM_LABELS)
print(f"Done updating dataframe keys.\n{80*'-'}\nCalculating state of charge "
      f"(x)...")
df["x"] = x_calculate(df, D_SAMPLE)
print(f"Done calculating state of charge (x).\n{80*'-'}\nPlotting voltage as a "
      f"function of state of charge...")
plot(df, D_SAMPLE, data_file.stem, plot_paths)
print(f"Done plotting voltage as a function of state of charge.\nPlease see "
      f"the {plot_folders} folders.\n{80*'-'}\nWriting .txt file...")
write(df, D_ECHEM_LABELS, data_file.name, txt_path)
print(f"Done writing .txt file.\nPlease see the '{txt_path.name}' folder.\n"
      f"{80*'-'}\nProgram done.")